# December 09, 2025

https://adventofcode.com/2025/day/9

In [3]:
def parse_text( text ):
    return [ [int(x) for x in line.split(",")] for line in text ]

In [4]:
test_text = f'''7,1
11,1
11,7
9,7
9,5
2,5
2,3
7,3'''

test = test_text.split("\n")
test = parse_text(test)


In [5]:
fn = "../data/2025/09.txt"
with open(fn, "r") as file:
    puzz_text = file.readlines()
puzz = [ line.strip() for line in puzz_text ]
puzz = parse_text(puzz)


# Part 1

In [6]:
def part1( puzz ):
    biggest = 0
    for i in range(len(puzz)):
        for j in range(i+1, len(puzz)):
            wid = abs(puzz[i][0]-puzz[j][0]) + 1
            hei = abs(puzz[i][1]-puzz[j][1]) + 1
            area = wid*hei
            if area > biggest:
                biggest = area
    return biggest

In [7]:
part1( test )

50

In [8]:
part1( puzz )

4773451098

# Part 2

## Attempt 1
Just plotting this thing for ideas!

In [22]:
def parse_pattern( puzz ):
    h = max( [tile[1] for tile in puzz] )
    w = max( [tile[0] for tile in puzz] )
    pattern = list()
    for _ in range(h+2):
        pattern.append( ["."] * (w+2) )

    for i, red_tile in enumerate(puzz):
        pattern[ red_tile[1] ][ red_tile[0] ] = "#"
        if i < len(puzz)-1:
            next_red = puzz[i+1]
        else:
            next_red = puzz[0]

        if red_tile[0] == next_red[0]:
            # same col, iterate row-wise
            x = min(red_tile[1], next_red[1])
            y = max(red_tile[1], next_red[1])
            for t in range(x+1,y):
                if pattern[t][red_tile[0]] == ".":
                    pattern[t][red_tile[0]] = "g"
        else:
            # same line, iterate colwise
            x = min(red_tile[0], next_red[0])
            y = max(red_tile[0], next_red[0])
            for t in range(x+1,y):
                if pattern[red_tile[1]][t] == ".":
                    pattern[red_tile[1]][t] = "g"

    return pattern

def print_pattern( pattern ):
    for line in pattern:
        print("".join([tile for tile in line]))




In [23]:
print_pattern(parse_pattern(test))

.............
.......#ggg#.
.......g...g.
..#gggg#...g.
..g........g.
..#gggggg#.g.
.........g.g.
.........#g#.
.............


In [24]:
# runs out of memory
# print_pattern(parse_pattern(puzz))

## Attempt 2
Fancy geometry

In [25]:
def orient_pattern( puzz ):
    '''reorient puzzle so that the first move is right, last move is up, and pattern starts on an outside corner'''
    # notation: x = line in text; y = column within that line
    # find starting tile - the leftmost tile on the highest line
    # (prioritizing higher over lefter)
    si = 0
    sx, sy = puzz[si]
    for i, tile in enumerate(puzz):
        if (tile[0] < sx) or (tile[0] == sx and tile[1] < sy):
            si = i
            sx, sy = tile

    # next tile is either down or right by construction
    if si < len(puzz) - 1:
        next_tile = puzz[si+1]
    else:
        next_tile = puzz[0]

    if next_tile[0] == sx:
        # tile is to the right... keep direction the same
        new_puzz = puzz[si:] + puzz[:si]
    else:
        # tile is down... reverse direction
        new_puzz = puzz[si::-1] + puzz[-1:si:-1]
    return new_puzz



In [63]:
def next_tile(puzz, i):
    if i == len(puzz) - 1:
        return puzz[0]
    else:
        return puzz[i+1]
    
def prev_tile(puzz, i):
    if i == 0:
        return puzz[-1]
    else:
        return puzz[i-1]
    
def dir(x, y):
    if x[0] == y[0]:
        if x[1] < y[1]:
            return "R"
        elif x[1] > y[1]:
            return "L"
        return None
    elif x[1] == y[1]:
        if x[0] < y[0]:
            return "D"
        elif x[0] > y[0]:
            return "U"
        return None
    else:
        if x[0] < y[0]:
            if x[1] < y[1]:
                return "RD"
            else:
                return "LD"
        else:
            if x[1] < y[1]:
                return "RU"
            else:
                return "LU"
def area( x, y ):
    return (abs(x[0]-y[0]) + 1) * (abs(x[1]-y[1])+1)

def best_rect( puzz, i, verbose=False ):
    tile = puzz[i]

    # get direction before and after tile i
    pt = prev_tile(puzz, i)
    nt = next_tile(puzz, i)
    pdir = dir( tile, pt ) # dir from tile to prev_tile
    ndir = dir( tile, nt ) # dir from tile to next_tile

    # canonical dir L/R then U/D
    if pdir in ["L", "R"]:
        cdir = pdir + ndir
    else:
        cdir = ndir + pdir

    # outside corner means that triangle
    # prev tile -> tile -> next tile is
    # part of the pattern for coords sufficiently
    # close to tile
    if pdir == "R":
        outside = ndir == "U"
    elif pdir == "D":
        outside = ndir == "R"
    elif pdir == "L":
        outside = ndir == "D"
    else:
        outside = ndir == "L"

    print( f'''Tile {i} [{tile[0]}, {tile[1]}]''')
    print( f'''Prev Tile [{pt[0]}, {pt[1]}] is from the {pdir}''')
    print( f'''Next Tile [{nt[0]}, {nt[1]}] is to the {ndir}''')
    print( f'''This is an {"outside" if outside else "inside"} corner with canonical direction {cdir}''' )
    
    best = area(tile, nt)
    # edge case to check the line between the last and first tile
    if i == 0:
        best = max(best, area(tile, puzz[-1]))

    for j in range(i+1, len(puzz)):
        rect_dir = dir( tile, puzz[j] )
        print(i,j,rect_dir)
        # for these cases, the rectangle lies outside the pattern
        if outside and cdir != rect_dir:
            continue
        if not outside and cdir == rect_dir:
            continue

        # look for any red tiles in the interior
        rectangle_is_good = True
        x0, x1 = min(tile[0], puzz[j][0]), max(tile[0], puzz[j][0])
        y0, y1 = min(tile[1], puzz[j][1]), max(tile[1], puzz[j][1])

        for k, red_tile in enumerate(puzz):
            print("Checking",k)
            if x0 < red_tile[0] and red_tile[0] < x1 and y0 < red_tile[1] and red_tile[1] < y1:
                rectangle_is_good = False
                print(f'''red_tile {k} is in the interior!''')
                break
        if rectangle_is_good:
            print(f'''rect is good between {i} and {j}''')
            best = max(best, area(tile, puzz[j]))
    # done testing all rectangles




        

In [56]:
test2 = orient_pattern( test )

In [57]:
test2

[[2, 3], [2, 5], [9, 5], [9, 7], [11, 7], [11, 1], [7, 1], [7, 3]]

In [64]:
best_rect(test2, 0)

Tile 0 [2, 3]
Prev Tile [7, 3] is from the D
Next Tile [2, 5] is to the R
This is an outside corner with canonical direction RD
0 1 R
0 2 RD
Checking 0
Checking 1
Checking 2
Checking 3
Checking 4
Checking 5
Checking 6
Checking 7
rect is good between 0 and 2
0 3 RD
Checking 0
Checking 1
Checking 2
Checking 3
Checking 4
Checking 5
Checking 6
Checking 7
rect is good between 0 and 3
0 4 RD
Checking 0
Checking 1
Checking 2
red_tile 2 is in the interior!
0 5 LD
0 6 LD
0 7 D


## Attempt 3
Check if the diagonal of the rectangle intersects any edge of the pattern   
Only count it if the lines cross all the way through

In [ ]:
# Turns out we don't need something so general
def lines_intersect(x0, y0, x1, y1):
    '''do lines segments from x0-y0 and from x1-y1 intersect?'''

    # x00 + t*(y00-x00) = x10 + s*(y10-x10)    <-- line matches
    # x01 + t*(y01-x01) = x11 + s*(y11-x11)    <-- col matches

    # t = (x10 - x00 + s*(y10-x10)) / (y00-x00)
    # x01 + (x10 - x00 + s*(y10-x10)) / (y00-x00) * (y01-x01) = x11 + s*(y11-x11)

    # if 0 <= s,t 1, the line segments intersect
    # if s_factor is 0 below, then one segment contains the other (maybe?)
    # these equations don't work if either line is horizontal

    lhs = x0[1] + (x1[0]-x0[0])/(y0[0]-x0[0])*(y0[1]-x0[1]) - x1[1]
    s_factor = (y1[1]-x1[1]) - (y1[0]-x1[0])/(y0[0]-x0[0])*(y0[1]-x0[1])
    print(s_factor)
    s = lhs / s_factor
    t = (x1[0] - x0[0] + s*(y1[0]-x1[0])) / (y0[0]-x0[0])
    
    intx0 = [ x0[0] + t*(y0[0]-x0[0]), x0[1] + t*(y0[1]-x0[1]) ]
    intx1 = [ x1[0] + s*(y1[0]-x1[0]), x1[1] + s*(y1[1]-x1[1]) ]
    print(intx0)
    print(intx1)

    return t, s


In [85]:
lines_intersect( [0,0], [1,1], [.5,10], [.5,0] )

-10.0
[0.5, 0.5]
[0.5, 0.5]


(0.5, 0.95)

In [83]:
lines_intersect( [0,0], [1,1], [-1,-1], [2,4] )

2.0
[-1.0, -1.0]
[-1.0, -1.0]


(-1.0, 0.0)

In [72]:
lines_intersect(test2[0], test2[3], test2[1], test2[2])

[5.5, 5.0]
[5.5, 5.0]


(0.5, 0.5)

## Attempt 4
Need to check all sides of rectangle for intersections

In [ ]:
def lines_intersect( x0, y0, x1, y1 ):
    if x0[0] == y0[0]:
        # line0 is horizontal
        if x1[0] == y1[0]:
            # two horizontal lines can't meaningfully intersect
            return False
        return (
            # vertical line spans where horizontal line is
            min(x1[0], y1[0]) < x0[0] and
            max(x1[0], y1[0]) > x0[0] and
            # horizontal line spans where vertical line is
            min(x0[1], y0[1]) < x1[1] and
            max(x0[1], y0[1]) < x1[1] 
        )
    
    # line0 is vertical
    if x1[1] == y1[1]:
        return False
    
    return (
            # vertical line spans where horizontal line is
            min(x0[0], y0[0]) < x1[0] and
            max(x0[0], y0[0]) > x1[0] and
            # horizontal line spans where vertical line is
            min(x1[1], y1[1]) < x0[1] and
            max(x1[1], y1[1]) < x0[1]
    )